# Cleaning and smoothing trajectory data with moving pandas

In [1]:
import movingpandas as mpd
import geopandas as gpd
import pandas as pd
from datetime import datetime, timedelta
from shapely.geometry import LineString, Point
from shapely.wkt import loads
import matplotlib.pyplot as plt
import os
from srai.regionalizers import geocode_to_region_gdf
from srai.datasets import AirbnbMulticityDataset
from srai.embedders import Hex2VecEmbedder
from srai.joiners import IntersectionJoiner
from srai.loaders.osm_loaders import OSMPbfLoader
from srai.loaders.osm_loaders.filters import HEX2VEC_FILTER
from srai.neighbourhoods.h3_neighbourhood import H3Neighbourhood
from srai.plotting import plot_regions
from srai.regionalizers import H3Regionalizer
import warnings
import torch

In [2]:
hvplot_defaults = {'tiles':'CartoLight', 'frame_height':320, 'frame_width':320, 'cmap':'Viridis', 'colorbar':True}
kwargs = {**hvplot_defaults, 'c':'speed', 'line_width':7}

In [3]:
gdf = gpd.read_parquet(os.path.join('output_data', 'geolife.parquet'))

In [4]:
gdf['time'] = pd.to_datetime(gdf['time'])
gdf.crs = 'EPSG:4326'

In [5]:
gdf.head()

,latitude,longitude,altitude,date,date_str,time,trajectory_id,mode,geometry,user_id
0,39.974294,116.399741,492.0,39816.056644,2009-01-03,2009-01-03 01:21:34,20090103012134,unknown,POINT (116.39974 39.97429),135
1,39.974292,116.399592,492.0,39816.056655,2009-01-03,2009-01-03 01:21:35,20090103012134,unknown,POINT (116.39959 39.97429),135
2,39.974309,116.399523,492.0,39816.056667,2009-01-03,2009-01-03 01:21:36,20090103012134,unknown,POINT (116.39952 39.97431),135
3,39.974320,116.399588,492.0,39816.056690,2009-01-03,2009-01-03 01:21:38,20090103012134,unknown,POINT (116.39959 39.97432),135
4,39.974365,116.399730,491.0,39816.056701,2009-01-03,2009-01-03 01:21:39,20090103012134,unknown,POINT (116.39973 39.97436),135


In [6]:
gdf.shape

(24876978, 10)

In [7]:
def validate_cords(gdf, lon_col='longitude', lat_col='latitude', trip_id_col='trajectory_id'):
    valid_lon = (-180 <= gdf[lon_col]) & (gdf[lon_col] <= 180)
    valid_lat = (-90 <= gdf[lat_col]) & (gdf[lat_col] <= 90)
    
    valid_coords = valid_lon & valid_lat
    
    invalid_trip_ids = gdf.loc[~valid_coords, trip_id_col].unique()
    
    gdf = gdf[~gdf[trip_id_col].isin(invalid_trip_ids)]
    
    return gdf

In [8]:
gdf = validate_cords(gdf)

In [9]:
pekin_area = geocode_to_region_gdf("Pekin, China")

In [10]:
gdf_pekin = gdf.sjoin(pekin_area)

In [11]:
gdf_pekin.shape

(19880914, 11)

In [12]:
gdf_merged = gdf.merge(gdf_pekin, how="left", indicator=True)

In [13]:
gdf_outside_pekin = gdf_merged[gdf_merged["_merge"] == "left_only"]

In [14]:
traj_outside_pekin = list(gdf_outside_pekin["trajectory_id"].unique())

In [15]:
gdf_pekin = gdf_pekin[~gdf_pekin["trajectory_id"].isin(traj_outside_pekin)]

In [16]:
traj_col = mpd.TrajectoryCollection(gdf_pekin,'trajectory_id', t = 'time', x = 'latitude', y = 'longitude')

In [17]:
traj_col

TrajectoryCollection with 16122 trajectories

In [19]:
traj_col.add_speed(units=("km", "h"), overwrite = True)

In [ ]:
# traj_col.add_timedelta(overwrite = True)

In [ ]:
# traj_col.add_direction(overwrite = True)

In [18]:
results = traj_col.to_point_gdf()
results.head()

,latitude,longitude,altitude,date,date_str,trajectory_id,mode,geometry,user_id,index_right
time,,,,,,,,,,
2000-01-01 23:12:19,39.988992,116.327023,128.937005,36526.966887,2000-01-01,20000101231219,unknown,POINT (116.32702 39.98899),163,"Beijing, China"
2000-01-01 23:13:21,39.990964,116.327041,221.128615,36526.967604,2000-01-01,20000101231219,unknown,POINT (116.32704 39.99096),163,"Beijing, China"
2000-01-01 23:15:23,39.993207,116.326827,217.191591,36526.969016,2000-01-01,20000101231219,unknown,POINT (116.32683 39.99321),163,"Beijing, China"
2007-04-12 09:31:32,39.974233,116.330383,823.490814,39184.396898,2007-04-12,20070412093132,unknown,POINT (116.33038 39.97423),142,"Beijing, China"
2007-04-12 09:39:37,39.974317,116.330450,823.490814,39184.402512,2007-04-12,20070412093132,unknown,POINT (116.33045 39.97432),142,"Beijing, China"


In [19]:
results.shape

(17371694, 10)

# Spatial Embedding

In [20]:
HEX_RES = 8

In [21]:
regionalizer = H3Regionalizer(resolution=HEX_RES)
regions = regionalizer.transform(pekin_area)

In [22]:
loader = OSMPbfLoader()
features = loader.load(regions, HEX2VEC_FILTER)

/Users/szymonleszkiewicz/anaconda3/envs/obsrai/lib/python3.10/site-packages/quackosm/osm_extracts/__init__.py:498: FutureWarning: `unary_union` returned None due to all-None GeoSeries. In future, `unary_union` will return 'GEOMETRYCOLLECTION EMPTY' instead.
  ].unary_union


In [23]:
joiner = IntersectionJoiner()
joint = joiner.transform(regions, features)

In [25]:
neighbourhood = H3Neighbourhood(regions)
embedder_hidden_sizes = [150, 100, 50, 10]
embedder = Hex2VecEmbedder(embedder_hidden_sizes)
device = "cuda" if torch.cuda.is_available() else "cpu"

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    embeddings = embedder.fit_transform(
        regions,
        features,
        joint,
        neighbourhood,
        trainer_kwargs={"max_epochs": 15, "accelerator": device},
        batch_size=100,
    )

100%|██████████| 32149/32149 [00:00<00:00, 43745.55it/s]
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 93.0 K
---------------------------------------
93.0 K    Trainable params
0         Non-trainable params
93.0 K    Total params
0.372     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=15` reached.


In [26]:
embeddings.to_parquet(os.path.join('output_data', f'embeddings_{HEX_RES}.parquet'))

# CLEAN

### max speed

In [23]:
cleaned_max = mpd.OutlierCleaner(traj_col).clean(
    v_max=300, units=("km", "h")
)

In [33]:
cleaned_max.trajectories[13].hvplot(**kwargs)+traj_col.trajectories[13].hvplot(**kwargs)

:Layout
   .Overlay.I  :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Path.I   :Path   [Longitude,Latitude]   (speed)
      .Points.I :Points   [Longitude,Latitude]   (speed,triangle_angle)
   .Overlay.II :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Path.I   :Path   [Longitude,Latitude]   (speed)
      .Points.I :Points   [Longitude,Latitude]   (speed,triangle_angle)

### alpha

In [23]:
cleaned_alpha = mpd.OutlierCleaner(traj_col).clean(alpha = 1.0)

In [25]:
cleaned_alpha.trajectories[12].hvplot(**kwargs)+traj_col.trajectories[12].hvplot(**kwargs)

:Layout
   .Overlay.I  :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Path.I   :Path   [Longitude,Latitude]   (speed)
      .Points.I :Points   [Longitude,Latitude]   (speed,triangle_angle)
   .Overlay.II :Overlay
      .WMTS.I   :WMTS   [Longitude,Latitude]
      .Path.I   :Path   [Longitude,Latitude]   (speed)
      .Points.I :Points   [Longitude,Latitude]   (speed,triangle_angle)

## SAVE

In [35]:
if not os.path.exists('output_data'):
    os.makedirs('output_data')

In [ ]:
# save cleaned_alpha and cleaned_max to point gdf and file
cleaned_max_gdf = cleaned_max.to_point_gdf()
cleaned_max_gdf.to_parquet(os.path.join('output_data', 'geolife_max.parquet'))

In [ ]:
cleaned_alpha_gdf = cleaned_alpha.to_point_gdf()
cleaned_alpha_gdf.to_parquet(os.path.join('output_data', 'geolife_alpha.parquet'))

### filter short trajectories

In [26]:
filtered_trajectory_collection = [
    trajectory
    for trajectory in cleaned_alpha.trajectories
    if trajectory.size() >= 10
]

In [30]:
cleaned = mpd.TrajectoryCollection(filtered_trajectory_collection)

# [Generalization](https://movingpandas.github.io/movingpandas-website/1-tutorials/7-generalizing-trajectories.html)

In [20]:
test = traj_col.trajectories[12] # 12 13
test.hvplot(**kwargs)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Path.I   :Path   [Longitude,Latitude]   (speed)
   .Points.I :Points   [Longitude,Latitude]   (speed,triangle_angle)

In [ ]:
test

In [31]:
traj_col_dp = mpd.DouglasPeuckerGeneralizer(cleaned).generalize(tolerance=0.0001)

In [32]:
results_dp = traj_col_dp.to_point_gdf()
results_dp.shape

(1199437, 11)

In [ ]:
i = 312
traj_col_dp.trajectories[i].hvplot(**kwargs)+traj_col.trajectories[i].hvplot(**kwargs)

# SMOOTH

In [ ]:
# smooth = mpd.KalmanSmootherCV(cleaned).smooth(process_noise_std=0.1, measurement_noise_std=10)
# smooth

In [ ]:
# smooth.trajectories[i].hvplot(**kwargs)+traj_col_dp.trajectories[i].hvplot(**kwargs)

In [33]:
smooth2 = mpd.KalmanSmootherCV(traj_col_dp).smooth(process_noise_std=0.1, measurement_noise_std=2)
smooth2

TrajectoryCollection with 15794 trajectories

In [ ]:
smooth.trajectories[i].hvplot(**kwargs)+smooth2.trajectories[i].hvplot(**kwargs)+traj_col_dp.trajectories[i].hvplot(**kwargs)

In [ ]:
i = 1645
smooth.trajectories[i].hvplot(**kwargs)+smooth2.trajectories[i].hvplot(**kwargs)+traj_col_dp.trajectories[i].hvplot(**kwargs)

In [34]:
smooth2_gdf = smooth2.to_point_gdf()
smooth2_gdf.shape

(1199437, 11)

In [35]:
if not os.path.exists('output_data'):
    os.makedirs('output_data')
smooth2_gdf.to_parquet(os.path.join('output_data', 'geolife_mpd2.parquet'))